In [2]:
import numpy as np
import pandas as pd
from load_data import load_train_data
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from keras.utils import to_categorical
from sklearn.preprocessing import MinMaxScaler
from gpu import set_gpu

set_gpu()

Num GPUs Available:  1
Using TensorFlow backend.


In [3]:
# Load data
feat,params,daq = load_train_data('train_data_raw_AB.mat')

## Run ANN

In [3]:
# Build NN model
model = Sequential([
    Dense(units=60, input_shape=(60,), activation='relu'),
    Dense(units=32, activation='relu'),
    Dense(units=7, activation='softmax')
])
model.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 60)                3660      
_________________________________________________________________
dense_1 (Dense)              (None, 32)                1952      
_________________________________________________________________
dense_2 (Dense)              (None, 7)                 231       
Total params: 5,843
Trainable params: 5,843
Non-trainable params: 0
_________________________________________________________________


In [4]:
# Index data
# grp = 1 (static), 2 (dynamic), 3 (0g), 4 (400g), 5 (500g), 6 (600g)
sub = 15
grp = 3
ind = (params[:,0] < sub) & (params[:,3] == grp)

# Shuffle and split data
feat_s, label_s = shuffle(feat[ind,:],params[ind,-2,np.newaxis])
X_train, X_test, y_train, y_test = train_test_split(feat_s, label_s, test_size=0.3, random_state=0, stratify=label_s)

# Reshape and scale data
y_train = np.squeeze(y_train-1)
scaler = MinMaxScaler(feature_range=(-1,1))
scaler.fit(X_train)
X_train_n = scaler.transform(X_train)
X_test_n = scaler.transform(X_test)

In [5]:
# Fit model
model.fit(x=X_train_n, y=y_train, validation_split=0.1, batch_size=32, epochs=30, verbose=2)

Epoch 1/30
965/965 - 1s - loss: 1.6391 - accuracy: 0.3937 - val_loss: 1.3493 - val_accuracy: 0.5644
Epoch 2/30
965/965 - 1s - loss: 1.2032 - accuracy: 0.6452 - val_loss: 1.0489 - val_accuracy: 0.6994
Epoch 3/30
965/965 - 1s - loss: 0.9757 - accuracy: 0.7238 - val_loss: 0.8801 - val_accuracy: 0.7356
Epoch 4/30
965/965 - 1s - loss: 0.8430 - accuracy: 0.7514 - val_loss: 0.7850 - val_accuracy: 0.7595
Epoch 5/30
965/965 - 1s - loss: 0.7647 - accuracy: 0.7677 - val_loss: 0.7273 - val_accuracy: 0.7679
Epoch 6/30
965/965 - 1s - loss: 0.7162 - accuracy: 0.7767 - val_loss: 0.6920 - val_accuracy: 0.7793
Epoch 7/30
965/965 - 1s - loss: 0.6828 - accuracy: 0.7845 - val_loss: 0.6617 - val_accuracy: 0.7828
Epoch 8/30
965/965 - 1s - loss: 0.6577 - accuracy: 0.7899 - val_loss: 0.6397 - val_accuracy: 0.7878
Epoch 9/30
965/965 - 1s - loss: 0.6370 - accuracy: 0.7948 - val_loss: 0.6211 - val_accuracy: 0.7948
Epoch 10/30
965/965 - 1s - loss: 0.6192 - accuracy: 0.8001 - val_loss: 0.6033 - val_accuracy: 0.7980

In [7]:
# Evaluate test set
out = np.argmax(model.predict(x=X_test_n), axis = 1)
np.sum(out.reshape(y_test.shape) == y_test - 1)/y_test.shape[0]

0.8455102040816327

## Run LDA

In [6]:
from lda import train_lda, predict

In [8]:
sub = 14
grp = 3
ind = (params[:,0] == sub) & (params[:,3] != grp)

# Shuffle and split data
X_test_new, y_test_new = shuffle(feat[ind,:],params[ind,-2,np.newaxis])
X_test_new = scaler.transform(X_test_new)
out = np.argmax(model.predict(x=X_test_new), axis = 1)
np.sum(out.reshape(y_test_new.shape) == y_test_new - 1)/y_test_new.shape[0]

0.7886285714285715

In [121]:
w,c = train_lda(X_train_n,y_train)
X_test_n = scaler.transform(X_test)
out = predict(X_test_n,w,c)
np.sum(out.reshape(y_test.shape) == y_test-1)/y_test.shape[0]

0.9933333333333333

## Run CNN

In [4]:
from load_data import process_daq


In [5]:
raw = process_daq(daq,params)
raw = raw[::2,:,:].transpose(2,1,0)

In [6]:
feat_s, label_s = shuffle(raw,params[:,-2])

In [7]:
X_train, X_test, y_train, y_test = train_test_split(feat_s, label_s, test_size=0.3, random_state=0, stratify=label_s)


In [8]:
scaler = MinMaxScaler(feature_range=(-1,1))
X_train = scaler.fit_transform(X_train.reshape(X_train.shape[0],-1)).reshape(X_train.shape)
X_test = scaler.transform(X_test.reshape(X_test.shape[0],-1)).reshape(X_test.shape)
# X_train = X_train.transpose(1,2,0)
# X_test = X_test.transpose(1,2,0)
X_train = X_train[...,np.newaxis]
X_test = X_test[...,np.newaxis]
y_train = np.squeeze(y_train-1)

In [9]:
model = Sequential([
    Conv2D(filters=32, kernel_size=(3,3), padding='same', input_shape=(6,100,1)),
    BatchNormalization(),
    Activation('relu'),
    MaxPool2D(pool_size=(2,2), strides=2),
    Conv2D(filters=32, kernel_size=(3,3), padding='same'),
    BatchNormalization(),
    Activation('relu'),
    MaxPool2D(pool_size=(2,2), strides=2),
    Flatten(),
    Dense(units=7, activation='softmax'),
])


In [10]:
model.summary()
model.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 6, 100, 32)        320       
_________________________________________________________________
batch_normalization (BatchNo (None, 6, 100, 32)        128       
_________________________________________________________________
activation (Activation)      (None, 6, 100, 32)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 3, 50, 32)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 3, 50, 32)         9248      
_________________________________________________________________
batch_normalization_1 (Batch (None, 3, 50, 32)         128       
_________________________________________________________________
activation_1 (Activation)    (None, 3, 50, 32)         0

In [11]:
model.fit(x=X_train, y=y_train, batch_size=16, validation_split=.1, epochs=10, verbose=2)

Epoch 1/10
11301/11301 - 25s - loss: 0.8935 - accuracy: 0.6905 - val_loss: 0.6401 - val_accuracy: 0.7875
Epoch 2/10
11301/11301 - 24s - loss: 0.6191 - accuracy: 0.7935 - val_loss: 0.5533 - val_accuracy: 0.8151
Epoch 3/10
11301/11301 - 24s - loss: 0.5472 - accuracy: 0.8198 - val_loss: 0.5149 - val_accuracy: 0.8262
Epoch 4/10
11301/11301 - 25s - loss: 0.5065 - accuracy: 0.8332 - val_loss: 0.4940 - val_accuracy: 0.8362
Epoch 5/10
11301/11301 - 25s - loss: 0.4802 - accuracy: 0.8424 - val_loss: 0.4633 - val_accuracy: 0.8460
Epoch 6/10
11301/11301 - 25s - loss: 0.4604 - accuracy: 0.8489 - val_loss: 0.4551 - val_accuracy: 0.8476
Epoch 7/10
11301/11301 - 25s - loss: 0.4449 - accuracy: 0.8539 - val_loss: 0.4521 - val_accuracy: 0.8496
Epoch 8/10
11301/11301 - 26s - loss: 0.4334 - accuracy: 0.8579 - val_loss: 0.4385 - val_accuracy: 0.8557
Epoch 9/10
11301/11301 - 25s - loss: 0.4239 - accuracy: 0.8618 - val_loss: 0.4265 - val_accuracy: 0.8611
Epoch 10/10
11301/11301 - 25s - loss: 0.4151 - accuracy